In [6]:
import urllib
import bs4
import re
import pandas as pd
from urllib.request import Request
import datetime
import time
import re
import codecs

In [19]:
## création d"une liste de dates 
liste_dates=[]
j = datetime.date(2020, 9, 30)
while j >= datetime.date(2020, 9, 10):
    liste_dates.append(str(j.strftime("%Y/%m/%d")))
    j = j + datetime.timedelta(days=-1)
print(len(liste_dates))
liste_dates[:1]

21


['2020/09/30']

In [20]:
# création de 2 listes l'une contient les articles, l'autre le jour de publication.
liste_titres = []
liste_jour_publication = []
liste_categorie =[]
liste_auteurs =[]
df_row = pd.DataFrame()
for d in liste_dates:
    ## exploration des pages internet en ajoutant la date à un début d'url
    status = 999
    compt = 1
    while status != 404:
        try:
            page_explorée = "https://www.lexpress.fr/archives/" + str(d)
            print(page_explorée)
            req_1 = Request(page_explorée,headers={'User-Agent': 'Mozilla/5.0'})
            request_text = request.urlopen(req_1).read()
            page = bs4.BeautifulSoup(request_text, "lxml")
            for titre in page.findAll("h3", {'class' : "title_mu"}):
                t = titre.get_text()
                print(t)
                liste_titres.append(t)
            for categorie in page.findAll("span", {'class' : "topic"}):
                cat = categorie.get_text()
                if len(cat)==0:
                    cat = ['']
                liste_categorie.append(cat)
                liste_jour_publication.append(d)   
            for group in page.findAll("div", {'class':"groups search_results"}):
                for url in group.findAll("a"):
                    url_article = url['href']
                    url_2 ="https:"+ url_article
                    req_2 = urllib.request.Request(url_2,headers={'User-Agent': 'Mozilla/5.0'})
                    request_text2 = request.urlopen(req_2).read()
                    page_article = bs4.BeautifulSoup(request_text2, "lxml")
                    tit=''
                    s=''
                    for div in page_article.findAll("div",{'class':"author__name"}):
                        for sp in div.findAll('span'):
                            for s in sp.findAll('span')[0]:
                                liste_auteurs.append(s) # ajout de la liste d'auteur associé à l'article    

            compt+=1  
        except Exception as e:
            status = 404
len(liste_titres)

https://www.lexpress.fr/archives/2020/09/30
Suez: Engie et Veolia d'accord sur les grandes lignes, se donnent jusqu'à lundi pour finaliser
Un lycéen de 16 ans blessé à coups de couteau en pleine classe en Seine-Saint-Denis
Covid-19 : 10 questions pour y voir plus clair dans la jungle des tests
Incendie de Notre-Dame : la Cour des comptes épingle la gestion des dons
Dégradation sanitaire à Paris, emplois supprimés chez Disney... Le point sur la pandémie
La "cancel culture", un phénomène qui inquiète
Thomas Chatterton Williams : "Comment peut-on remettre en question l'idéal républicain ?"
Génocide au Rwanda: Félicien Kabuga sera bien remis à la justice internationale
Procès des attentats de 2015: le puzzle "complexe" des armes et des intermédiaires
"Nous défendons l'oeuvre de J. K. Rowling, mais aussi sa liberté d'expression"
Après un débat chaotique, des démocrates appellent Biden à refuser les suivants (en vain)
La jeune Victorine semble être décédée d'une noyade "non accidentelle"
Att

https://www.lexpress.fr/archives/2020/09/27
Sénatoriales: la majorité de droite "confortée", un groupe écologiste annoncé
Très attendues, les enchères sur la 5G débutent mardi
Attaque au hachoir à Paris: six gardes à vue en cours, une vidéo à l'étude
Liban : comment les partis chiites peuvent faire échouer l'initiative française
Loukachenko "doit partir" assure Macron, avant une nouvelle manifestation en Biélorussie
"Restons ouverts": les patrons de bars manifestent à Paris
Covid-19: que risquent les établissements s'ils refusent de se plier aux nouvelles règles?
Bars, piscines, salles de sport : le point sur ce qui va fermer et ce qui va rester ouvert
Attaque à Paris : comment sécuriser les lieux visés par des actes terroristes ?
Attaque à Paris : "Youssef", passé de deuxième suspect à "héros" remercié
Philippe Martinez (CGT) demande le retrait du schéma national de maintien de l'ordre
Covid-19: des médecins réclament "des mesures drastiques"
Covid-19 : "C'est maintenant qu'il faut pr

https://www.lexpress.fr/archives/2020/09/23
Coronavirus : plus de cinq millions de cas en Europe, la France durcit les restrictions
Décès de Pierre Troisgros, légendaire chef trois étoiles de Roanne
L'UE lance une réforme de l'asile et va devoir vaincre les réticences
Covid-19 : une deuxième vague en Europe pourrait durer jusqu'en janvier 2021
Covid-19 : un prélèvement sanguin pour prédire le risque de mortalité des patients
Les médias se mobilisent pour soutenir Charlie Hebdo, de nouveau menacé
L'icône de la chanson française Juliette Gréco est morte
2020, année de tous les records pour le vélo
L'été 2020, le plus sec depuis le début des mesures, selon Météo-France
Le chef Pierre Troisgros, clé de voûte d'un succès familial
Macron annonce "sept jours obligatoires" dans le congé paternité d'un mois
"577 abeilles" contre les néonicotinoïdes
Sibeth Ndiaye sur les masques : "A aucun moment, on ne m'a demandé de mentir"
La France demande à l'Italie d'accueillir le navire humanitaire Alan K

https://www.lexpress.fr/archives/2020/09/20
Les Journées du Patrimoine ont attiré moins de monde
Retour progressif à la normale après des intempéries dans le Gard, un deuxième disparu
"Poutine, retire ta fourchette" : À Minsk, des milliers de Biélorusses toujours mobilisés
Covid-19 : comment les autres pays se préparent à une deuxième vague
Les rendez-vous secrets de Castex avec Fillon, Valls, Philippe et même... Bertrand
Dupond-Moretti justifie son enquête administrative contre des magistrats du PNF
Cour suprême des Etats-Unis : qui sont les favorites pour succéder à Ruth Bader Ginsburg ?
Dupond-Moretti justifie son enquête administrative contre des magistrats du PNF
Etats-Unis : Quelles sont les conséquences d'un bannissement de TikTok et WeChat ?
Pourquoi les États-Unis imposent-ils unilatéralement de nouvelles sanctions à l'Iran ?
En Thaïlande, le mouvement pro-démocratie ne faiblit pas
Le choix de L'Express : Benoît Peeters
L'Iran souligne "l'isolement" de Washington après son ann

https://www.lexpress.fr/archives/2020/09/17
Rentrée syndicale sous Covid avec des cortèges masqués dans plusieurs villes françaises
Covid-19 : plus de 10 000 nouveaux cas et 50 décès en 24 heures
L'usine Bridgestone de Béthune menacée
"De grosses difficultés à venir": en réanimation, la pression du Covid revient
Agressions sexuelles : les femmes qui accusent Donald Trump
Nouvelles restrictions à Lyon, port du masque en crèches : les annonces d'Olivier Véran
L'EI revendique l'assassinat des six Français au Niger
Étude: un spray nasal serait capable "d'inactiver le Sars-CoV-2" et de limiter l'infection
Raphaël Enthoven : "Quand Edouard Louis raconte sa vie, nul ne le lui reproche"
Voiture, vélo, marche : comment se déplacent les Français ?
Bridgestone : du côté des politiques, des mots forts, mais des solutions encore floues
Des stocks au plus bas, une collecte difficile: la France manque de sang
Air France, Renault, Nokia, Auchan... La liste des plans sociaux s'allonge
INFOGRAPHIES. Six

https://www.lexpress.fr/archives/2020/09/14
Brexit: le texte controversé de Boris Johnson franchit un premier obstacle au Parlement
Au procès Charlie, le deuil sans fin des proches du policier Ahmed Merabet
Oracle bien parti pour reprendre les activités américaines de TikTok
#Lundi14Septembre : le hashtag qui dénonce le sexisme dans les lycées de France
L'ancien président Valéry Giscard d'Estaing hospitalisé à Paris
Ce trop-plein d'épargne qui fait saliver l'exécutif
Valéry Giscard d'Estaing hospitalisé à Paris
Présidentielle américaine : à J-50, un Donald Trump mal engagé mais pas encore perdant
Paris: un adjoint démissionne suite à des accusations de harcèlement sexuel
Transports: le "coup de pouce" vélo prolongé, avec 20 millions en plus
Repas pour les migrants : Véran a envisagé d'aller défier Darmanin à Calais
Castex met la pression sur les entreprises afin de sauver l'emploi
Turquie-Grèce : faut-il croire à un apaisement des tensions en Méditerranée ?
Roger-Pol Droit : Supprimer 

https://www.lexpress.fr/archives/2020/09/11
Le Conseil d'Etat suspend la chasse à la tourterelle des bois
Procès Charlie Hebdo: "je voulais dire aux familles qu'elles sont très courageuses"
"Singes" et "nègres": Sarkozy s'attire les foudres de la gauche
Covid-19: face à une "dégradation", le gouvernement opte pour une approche locale
Profs inquiets, médecins débordés : faut-il un protocole sanitaire pour les enfants?
Al Qaïda menace de nouveau Charlie Hebdo pour avoir réédité des caricatures de Mahomet
Covid-19 : pour Jean Castex, un discours de moins de 10 minutes et peu de concret
LVMH-Tiffany : ce que l'Américain reproche au géant du luxe français
"Ils se laissent mourir": la solitude, deuxième épidémie qui inquiète les gérants d'Ehpad
Dépistage prioritaire, traçage... Ce qu'il faut retenir des annonces de Jean Castex
Covid-19 : accélération en Europe, ralentissement en Amérique... Le point sur la pandémie
VIDEO. Test antigénique : "Ce n'est pas un test qui va remplacer un autre"
VI

912

In [21]:
print(len(liste_titres))
print(len(liste_categorie))
print(len(liste_jour_publication))
print(len(liste_auteurs))

912
912
912
15


In [22]:
df = pd.DataFrame({ 'titre':liste_titres,'jour_publication':liste_jour_publication,'categorie':liste_categorie})
df

,titre,jour_publication,categorie
0,Suez: Engie et Veolia d'accord sur les grandes...,2020/09/30,Société
1,Un lycéen de 16 ans blessé à coups de couteau ...,2020/09/30,Société
2,Covid-19 : 10 questions pour y voir plus clair...,2020/09/30,Santé
3,Incendie de Notre-Dame : la Cour des comptes é...,2020/09/30,Economie
4,"Dégradation sanitaire à Paris, emplois supprim...",2020/09/30,Monde
...,...,...,...
907,Salariés contraints de garder leurs enfants : ...,2020/09/10,Société
908,"""C'est une sorte de triumvirat qui la prend en...",2020/09/10,Droit du travail
909,"Covid-19: plus de 900.000 morts, les essais de...",2020/09/10,Monde
910,Brexit: réunion d'urgence UE-GB pour clarifier...,2020/09/10,Monde


In [23]:
df.to_csv('csv/titres_Lexpress4.csv', index=False)